# Description

This notebook is to generate known good outputs (KGOs) for solving Naghavi model after we have added a cycle step size argument to the `advance_cycle()` function of the solver.

With different step sizes, there are slightly different solutions reached, so tests need these different KGOs to have as comparisons.

In [8]:
import numpy as np
import json
import pathlib
import os

from ModularCirc.Solver import Solver

from ModularCirc.Models.NaghaviModel import NaghaviModel
from ModularCirc.Models.NaghaviModelParameters import NaghaviModelParameters

In [9]:
TIME_SETUP_DICT = {
        'name'       :  'TimeTest', # the name asssociated with the temporal discretization (not super important.. names internat variable)
        'ncycles'    :  30,         # the maximum number of cycles for which we run the simulation
        'tcycle'     :  1000.,      # the duration of a heart beat (here in ms)
        'dt'         :  1.0,        # the duration of a discrete time step
        'export_min' :  2           # number of time steps for which we export the simulation (can also be used as a way to impose a minimum number of pulses)
    }

model = NaghaviModel(
    time_setup_dict=TIME_SETUP_DICT,
    suppress_printing=True
    )

In [12]:
# Start the loop with different cycle step size to generate JSON files for each loop:
cycle_step_sizes = [1, 3, 5, 7]
# cycle_step_sizes = [5]

# Dictionary to store the results for all cycle step sizes
all_results = {
    "metadata": {
        "description": "Results for the Naghavi model for different cycle_step_size values",
        "cycle_step_sizes": cycle_step_sizes
    },
    "results": {}
}

# Loop through each cycle step size
for cycle_step_size in cycle_step_sizes:

    print(cycle_step_size)

    model = NaghaviModel(
        time_setup_dict=TIME_SETUP_DICT,
        suppress_printing=True
        )

    # Initializing the solver
    solver = Solver(model=model)

    # Solver is being setup: switching off console printing and setting the solver method to "LSODA"  
    solver.setup(
        optimize_secondary_sv=False,
        suppress_output=True,
        step_tol=0.001,
        conv_cols=['p_lv', 'v_lv'],
        method='LSODA',
        step=cycle_step_size
    )    

    # Running the model
    solver.solve()

    # Define the indexes of the equivalent to the last cycles
    tind_fin = np.arange(start=model.time_object.n_t-model.time_object.n_c,
                          stop=model.time_object.n_t)

    # Dictionary to store the final cycle values for each component for the current step size
    final_cycle_values = {}

    # From each of the components, retrieve the volume (<V>), pressure (<P_i>) and flow (<Q_i>)
    for key, value in model.components.items():
        final_cycle_values[key] = {
            'V': value.V.values[tind_fin].mean(),
            'P_i': value.P_i.values[tind_fin].mean(),
            'Q_i': value.Q_i.values[tind_fin].mean()
        }

    # Add the results for the current cycle step size to the main dictionary
    all_results["results"][cycle_step_size] = final_cycle_values

# Get the base directory of the current file
base_dir = str(pathlib.Path().resolve())

output_file_path = os.path.join('../../','tests','expected_outputs','NaghaviModel_expected_output.json')

# Save all results to a single JSON file in the tests/expected_outputs directory
with open(output_file_path, 'w') as f:
    json.dump(all_results, f, indent=4)

1
3
5
7
